In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from scipy.stats.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-features/"))

['test_features_c.pkl', 'test_features_w.pkl', 'train_features_h.pkl', 'test_features_h.pkl', 'train_features_r.pkl', 'train_features_c.pkl', 'train_features_s.pkl', 'test_features_r.pkl', 'train_features_w.pkl', 'test_features_s.pkl']


In [2]:
train_df = pd.read_pickle('../input/mini-model-data/train_df_c.pkl')
test_df = pd.read_pickle('../input/mini-model-data/test_df_c.pkl')

train_identity = pd.read_pickle('../input/mini-model-data/train_identity.pkl')
test_identity = pd.read_pickle('../input/mini-model-data/test_identity.pkl')

train = train_df.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_df.merge(test_identity, how='left', left_index=True, right_index=True)

train_y = train_df['isFraud'].copy()

# print(train_df.shape)
# print(test_df.shape)
# print(train_identity.shape)
# print(test_identity.shape)
print(train.shape)
print(test.shape)

del train_df, test_df, train_identity, test_identity

(68519, 433)
(69266, 433)


In [4]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987010,0,86549,75.875000,C,16496,352.0,117.0,mastercard,134.0,credit,NaN,NaN,NaN,NaN,gmail.com,gmail.com,1.0,4.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,M0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,2.0,1.0,2.0,2.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.3125,31.84375,90.3125,90.3125,90.3125,90.3125,0.0,0.0,0.0,75.887497,75.887497,75.887497,3.0,3.0,3.0,3.0,4.0,4.0,2.0,2.0,2.0,0.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.3125,90.3125,90.3125,0.0,90.3125,90.3125,90.3125,

In [5]:
target_encode_book = [
    'C1', 'C2', 'C4', 'C6', 'C7', 'C8', 'C10', 'C11', 'C12','C13', 'C14', 
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'D9',
    'device_name', 'DeviceType',
    'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
    'id_20', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'lastest_browser', 
     'id_36', 'id_37',
    'P_emaildomain', 'R_emaildomain', 'cents'
]
# CV: C13, C14

In [6]:
def fe_dt(df):
    
    df['DT'] = df['TransactionDT'].apply(
        lambda x: (START_DATE + datetime.timedelta(seconds=x)))  # tmp    
    df['DT_month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_week_year'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_day_year'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    
    for col in ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8','D10','D12', 'D13','D14', 'D15']:
        # day map
        day_temp_dict = df.groupby('DT_day_year')[col].agg(['mean']).rename(columns={'mean':col + '_day_mean'})
        day_temp_dict = day_temp_dict[col + '_day_mean'].to_dict()
        df[col + '_day_mean'] = df['DT_day_year'].map(day_temp_dict)
        df[col + '_day_mean'] = df[col] - df[col + '_day_mean']
        
        # week map
        week_temp_dict = df.groupby('DT_week_year')[col].agg(['mean']).rename(columns={'mean':col + '_week_mean'})
        week_temp_dict = week_temp_dict[col + '_week_mean'].to_dict()
        df[col + '_week_mean'] = df['DT_week_year'].map(week_temp_dict)
        df[col + '_week_mean'] = df[col] - df[col + '_week_mean']

    df = df.drop(['DT'],axis=1)
    
    return df

In [7]:
def fe_uid(train_df, test_df):
    
    train_df['uid'] = train_df['card1'].astype(str)+'_'+train_df['card2'].astype(str)
    test_df['uid'] = test_df['card1'].astype(str)+'_'+test_df['card2'].astype(str)

    train_df['uid2'] = train_df['uid'].astype(str)+'_'+train_df['card3'].astype(str)+'_'+train_df['card5'].astype(str)
    test_df['uid2'] = test_df['uid'].astype(str)+'_'+test_df['card3'].astype(str)+'_'+test_df['card5'].astype(str)

    train_df['uid3'] = train_df['uid2'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'+train_df['addr2'].astype(str)
    test_df['uid3'] = test_df['uid2'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'+test_df['addr2'].astype(str)

    # Check if the Transaction Amount is common or not (we can use freq encoding here)
    # In our dialog with a model we are telling to trust or not to these values   
    train_df['TransactionAmt_check'] = np.where(train_df['TransactionAmt'].isin(test_df['TransactionAmt']), 1, 0)
    test_df['TransactionAmt_check']  = np.where(test_df['TransactionAmt'].isin(train_df['TransactionAmt']), 1, 0)

    i_cols = ['card1','card2','card3','card5','uid','uid2','uid3']

    for col in i_cols:
        for agg_type in ['mean','std']:
            new_col_name = col+'_TransactionAmt_'+agg_type
            temp_df = pd.concat([train_df[[col, 'TransactionAmt']], test_df[[col,'TransactionAmt']]])
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            train_df[new_col_name] = train_df[col].map(temp_df)
            test_df[new_col_name]  = test_df[col].map(temp_df)

    train_df['TransactionAmt'] = np.log1p(train_df['TransactionAmt'])
    test_df['TransactionAmt'] = np.log1p(test_df['TransactionAmt'])   
    
    train_df = train_df.drop(['uid','uid2','uid3'],axis=1)
    test_df = test_df.drop(['uid','uid2','uid3'],axis=1)
    
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)
    
    return train_df, test_df

In [8]:
# feature engineering

def fe(train, test):

    drop_na_cols = []
    for col in train.columns:
        if (len(train[col].dropna().unique()) <= 1):
            drop_na_cols.append(col)
    print(len(drop_na_cols), ' features are removed...')
    # remove all nan columns
    train = train.drop(drop_na_cols, axis=1)
    test = test.drop(drop_na_cols, axis=1)
    
    # user id
    train, test = fe_uid(train, test)

    ignore_cols = ['P_emaildomain', 'R_emaildomain', 'DeviceInfo', 'id_30', 'id_31', 'id_33']
    for col in train.columns:
        if (train[col].dtype == 'object' or test[col].dtype == 'object') and col not in ignore_cols:
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[col].values) + list(test[col].values))
            train[col] = lbl.transform(list(train[col].values))
            test[col] = lbl.transform(list(test[col].values))
    
    for df in [train, test]:
        # device info
        df = setDevice(df)
        
#         # id_30
#         df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
#         df['id_30_device'] = df['id_30'].apply(
#             lambda x: ''.join([i for i in x if i.isalpha()]))
#         df['id_30_version'] = df['id_30'].apply(
#             lambda x: ''.join([i for i in x if i.isnumeric()]))
        
        # id_31
        df = setBrowser(df)
        
        # nan encoding
        for col in train.columns:
            if(train[col].isnull().sum() / len(train) > 0.2):
                df[col + '_isnull'] = df[col].isnull() + 0
        
        # email map
        for item in ['P_emaildomain', 'R_emaildomain']:
            df[item] = df[item].map(emails)
            
#         # id_33
#         df['id_33'] = df['id_33'].fillna('0x0')
#         df['id_33_0'] = df['id_33'].apply(lambda x: x.split('x')[0]).astype(int)
#         df['id_33_1'] = df['id_33'].apply(lambda x: x.split('x')[1]).astype(int)
#         df['id_33'] = np.where(df['id_33']=='0x0', np.nan, df['id_33'])
#         df['id_33_area'] = df['id_33_0'] * df['id_33_1']
#         df['id_33_ratio'] = df['id_33_0'] / df['id_33_1']
        
        # cents
        df['cents'] = df['TransactionAmt'].apply(lambda x: x - int(x))
        

    train['cents'] = train['TransactionAmt'].apply(lambda x: x - int(x))
    test['cents'] = test['TransactionAmt'].apply(lambda x: x - int(x))
    
    for col in target_encode_book:
        train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)
        
        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
        train[col+'_fq_enc'] = train[col].map(fq_encode)
        test[col+'_fq_enc']  = test[col].map(fq_encode)

    un_corr_cols = []
    for col in train.columns:
        if (train[col].dtype != 'object'):
            if(abs(pearsonr(train[col].fillna(-999), train_y)[1]) > 0.05):
                un_corr_cols.append(col)
            print(col, pearsonr(train[col].fillna(-999), train_y))

    train = fe_dt(train)
    test = fe_dt(test)
    
    train = train.drop(target_encode_book + un_corr_cols, axis=1)
    test = test.drop(target_encode_book + un_corr_cols, axis=1)
    
    return train, test, drop_na_cols + un_corr_cols


train, test, drop_cols = fe(train, test)

102  features are removed...
isFraud (1.0, 0.0)
TransactionDT (0.045559730787131436, 8.085162577449925e-33)
TransactionAmt (0.05621127737486544, 4.4340778406425145e-49)
card1 (-0.045008945044869804, 4.5353480181983014e-32)
card2 (-0.12620665583309087, 3.104178102746573e-241)
card3 (0.027867240226590085, 2.967796337502016e-13)
card4 (0.012264582404298846, 0.0013252468842192904)
card5 (-0.006789339292504178, 0.07553951373886221)
card6 (-0.1323545905179257, 2.620060006280923e-265)
addr1 (-0.025361944595314583, 3.142722410029816e-11)
addr2 (-0.02493604777146377, 6.65695549352958e-11)
dist2 (0.009413689553987784, 0.013733875749536646)
C1 (0.024862063416616276, 7.574626683895987e-11)
C2 (0.03083725660382484, 6.815778539306675e-16)
C4 (0.01278755247642276, 0.000815843976822108)
C6 (0.01278755247642276, 0.000815843976822108)
C7 (0.012862502632384138, 0.000759951547216355)
C8 (0.01655539540299817, 1.465703653068633e-05)
C10 (0.010010836036373087, 0.008780924208169795)
C11 (0.018975375713745503,

V247 (0.05695941271845002, 2.384332821943026e-50)
V248 (0.0569852351709434, 2.1540332621929484e-50)
V249 (0.05697329129131358, 2.2576651472824847e-50)
V250 (0.05156063093313956, 1.4548685735523425e-41)
V251 (0.051578240176748336, 1.3664165270658522e-41)
V252 (0.05692501600368881, 2.7295963765898368e-50)
V253 (0.05693377635508676, 2.6372038736688626e-50)
V254 (0.056928850317998034, 2.688767111405048e-50)
V255 (0.05154444733646961, 1.5411710880719245e-41)
V256 (0.05155107507174684, 1.5052282141892146e-41)
V257 (0.058413269862385694, 7.287310999921673e-53)
V258 (0.05873004094910815, 2.0241380761160566e-53)
V259 (0.052834549471159134, 1.4731659805732478e-43)
V260 (0.05705893810857524, 1.611499990611012e-50)
V261 (0.05729943503830008, 6.235661307853419e-51)
V262 (0.05718176171155375, 9.927899945224316e-51)
V263 (0.12940327737886692, 1.3253663482634723e-253)
V264 (0.15740305508946983, 0.0)
V265 (0.14652542378789268, 0.0)
V266 (0.055299011763416464, 1.4862323992339602e-47)
V267 (0.05605279070

id_07_isnull (-0.019720244095931646, 2.4379666901374643e-07)
id_08_isnull (-0.019720244095931646, 2.4379666901374643e-07)
id_09_isnull (-0.16405962811991853, 0.0)
id_10_isnull (-0.16405962811991853, 0.0)
id_14_isnull (0.037842526012088616, 3.8013504529952244e-23)
id_18_isnull (0.0015545085313965895, 0.6840791873585494)
id_21_isnull (-0.019390214712755725, 3.854770898007371e-07)
id_22_isnull (-0.019390214712755725, 3.854770898007371e-07)
id_24_isnull (-0.041012835868684844, 6.611769227592544e-27)
id_25_isnull (-0.01922022331755327, 4.866967466078913e-07)
id_26_isnull (-0.0189354923960608, 7.161434583769899e-07)
cents (0.002715046846380384, 0.47728091366591824)
C1_target_code (0.36970896731905345, 0.0)
C1_fq_enc (-0.35944555664707917, 0.0)
C2_target_code (0.19991311082257834, 0.0)
C2_fq_enc (-0.21533668034761205, 0.0)
C4_target_code (0.3183986052570076, 0.0)
C4_fq_enc (-0.2858477389586836, 0.0)
C6_target_code (0.3183538398233523, 0.0)
C6_fq_enc (-0.28584564636524495, 0.0)
C7_target_code 

In [9]:
# col = 'DeviceInfo'
# train[col+'_target_code'], test[col+'_target_code'] = target_encode(train[col], test[col], train_y,min_samples_leaf=100, noise_level=0.01)

# plt.scatter(train[train.isFraud==0]['TransactionDT'], train[train.isFraud==0][col+'_target_code'], c='b')
# plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col+'_target_code'], c='r')
# plt.scatter(test['TransactionDT'], test[col+'_target_code'], c='g')

In [10]:
train.head()

,isFraud,TransactionDT,TransactionAmt,addr1,addr2,dist2,D1,D2,D3,D4,D5,D6,D7,D8,D14,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V50,V51,V52,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V66,V67,V71,V72,V73,V74,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V92,V93,V95,V96,V97,V101,V102,V103,V123,V124,V125,V126,V127,V128,V132,V133,V134,V135,V136,V137,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V284,V285,V287,V291,V294,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_21,id_31,id_38,DeviceInfo,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,addr1_isnull,addr2_isnull,dist2_isnull,D2_isnull,D3_isnull,D5_isnull,D7_isnull,D8_isnull,D9_isnull,D14_isnull,V217_isnull,V218_isnull,V219_isnull,V223_isnull,V224_isnull,V225_isnull,V228_isnull,V229_isnull,V230_isnull,V231_isnull,V232_isnull,V233_isnull,V235_isnull,V236_isnull,V237_isnull,V240_isnull,V241_isnull,V242_isnull,V243_isnull,V244_isnull,V246_isnull,V247_isnull,V248_isnull,V249_isnull,V252_isnull,V253_isnull,V254_isnull,V257_isnull,V258_isnull,V260_isnull,V261_isnull,V262_isnull,V263_isnull,V264_isnull,V265_isnull,V266_isnull,V267_isnull,V268_isnull,V269_isnull,V273_isnull,V274_isnull,V275_isnull,V276_isnull,V277_isnull,V278_isnull,id_03_isnull,id_04_isnull,id_07_isnull,id_08_isnull,id_09_isnull,id_10_isnull,id_14_isnull,id_21_isnull,id_22_isnull,id_24_isnull,id_25_isnull,id_26_isnull,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C4_target_code,C4_fq_enc,C6_target_code,C6_fq_enc,C7_target_code,C7_fq_enc,C8_target_code,C8_fq_enc,C10_target_code,C10_fq_enc,C11_target_code,C11_fq_enc,C12_target_code,C12_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card3_fq_enc,card4_target_code,card4_fq_enc,card5_target_code,card5_fq_enc,card6_target_code,card6_fq_enc,D9_target_code,D9_fq_enc,device_name_target_code,device_name_fq_enc,DeviceType_target_code,DeviceType_fq_enc,id_13_target_code,id_13_fq_enc,id_14_target_code,id_14_fq_enc,id_15_target_code,id_15_fq_enc,id_16_target_code,id_16_fq_enc,id_17_target_code,id_17_fq_enc,id_18_target_code,id_19_target_code,id_20_target_code,id_22_target_code,id_22_fq_enc,id_23_target_code,id_23_fq_enc,id_24_target_code,id_24_fq_enc,id_25_fq_enc,id_26_target_code,id_26_fq_enc,id_27_target_code,id_27_fq_enc,id_28_target_code,id_28_fq_enc,id_29_target_code,id_29_fq_enc,lastest_browser_target_code,lastest_browser_fq_enc,id_36_target_code,id_36_fq_enc,id_37_target_code,id_37_fq_enc,P_emaildomain_target_code,P_emaildomain_fq_enc,R_emaildomain_target_code,R_emaildomain_fq_enc,cents_target_code,cents_fq_enc,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,D1_day_mean,D1_week_mean,D2_day_mean,D2_week_mean,D3_day_mean,D3_week_mean,D4_day_mean,D4_week_mean,D5_day_mean,D5_week_mean,D6_day_mean,D6_week_mean,D7_day_mean,D7_week_mean,D8_day_mean,D8_week_mean,D10_day_mean,D10_week_mean,D12_day_mean,D12_week_mean,D13_day_mean,D13_week_mean,D14_day_mean,D14_week_mean,D15_day_mean,D15_week_mean
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [11]:
train.to_pickle('../input/mini-model-features/train_features_c.pkl')
test.to_pickle('../input/mini-model-features/test_features_c.pkl')

# POINT_SIZE = 2
# for col in train.columns:
#     fig = plt.figure(figsize=(12,8))

#     plt.scatter(train[train['DT_month'] == 12]['TransactionDT'], train[train['DT_month'] == 12][col], s=POINT_SIZE, c='b')     
#     plt.scatter(train[train['DT_month'] == 13]['TransactionDT'], train[train['DT_month'] == 13][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 14]['TransactionDT'], train[train['DT_month'] == 14][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 15]['TransactionDT'], train[train['DT_month'] == 15][col], s=POINT_SIZE, c='k')
#     plt.scatter(train[train['DT_month'] == 16]['TransactionDT'], train[train['DT_month'] == 16][col], s=POINT_SIZE, c='b')
#     plt.scatter(train[train['DT_month'] == 17]['TransactionDT'], train[train['DT_month'] == 17][col], s=POINT_SIZE, c='k')

#     plt.scatter(train[train.isFraud==1]['TransactionDT'], train[train.isFraud==1][col], s=POINT_SIZE * 3, c='r')
    
#     plt.scatter(test['TransactionDT'], test[col], s=POINT_SIZE, c='g')
#     plt.title(col)
#     plt.grid()
# #     plt.show()
    
#     fig.savefig("../eda/mini-models/C/C_" + str(col) + ".png")
#     plt.cla()
#     plt.clf()
#     plt.close(fig)
